## Import libraries

In [4]:
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

## Read object-feature matrix

In [6]:
data_wisdm = pd.read_table("../data/WISDM/WISDM_ar_v1.1_raw_cleared.txt", delimiter=',')
data_wisdm.columns = ['id_user', 'activity', 'timestamp', 'x', 'y', 'z']

data_uschad = pd.read_table("../data/USC-HAD/USC-HAD_cleared.txt", delimiter=',')
data_uschad.columns = ['id_user', 'activity', 'timestamp', 'x', 'y', 'z']

In [7]:
df_expert_wisdm = pd.read_csv("../data/features/expert_wisdm.csv")
df_ar_wisdm = pd.read_csv("../data/features/ar_wisdm.csv")
df_ssa_wisdm = pd.read_csv("../data/features/ssa_wisdm.csv")

In [8]:
df_expert_uschad = pd.read_csv("../data/features/expert_uschad.csv")
df_ar_uschad = pd.read_csv("../data/features/ar_uschad.csv")
df_ssa_uschad = pd.read_csv("../data/features/ssa_uschad.csv")

# Classification

In [9]:
def get_internal_score(clf, X, y, max_iter=25):
    nb = np.unique(y).shape[0]
    scores = np.zeros(nb+1)
    for j in range(max_iter):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)
        scores[0] += accuracy_score(y_test, y_predict)
        for i in range(nb):
            scores[i+1] += accuracy_score(1*(np.array(y_test) == i), 
                                          1*(np.array(y_predict) == i))
            
    return scores / max_iter

In [10]:
def get_score(df, estimator, params_grid, test_size=0.3):
    X = df.iloc[:, 1:].values
    y = df['activity'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    clf = GridSearchCV(estimator, params_grid)
    clf.fit(X_train, list(y_train))
    clf_lr = clf.best_estimator_
    scores = get_internal_score(clf_lr, X, list(y))
    
    return scores

## Testing part 

In [32]:
parameters_lr = {'lr__penalty': ['l1', 'l2'], 
                 'lr__class_weight': ['balanced', None], 
                 'lr__C': 10. ** np.arange(-1, 4, 1)}
parameters_svm = {'svc__kernel': ['rbf'], 
                  'svc__C': 10. ** np.arange(-1, 4, 1), 
                  'svc__gamma': 10. ** np.arange(-3, 2, 1),
                  'svc__class_weight': ['balanced', None]}

parameters_rf = {'n_estimators': [200], 
                 'class_weight': ['balanced', None], 
                 'max_depth': [None, 3, 5, 11]}

scores_wisdm = {}
scores_uschad = {}

In [36]:
lr_pipeline = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression())])
svc_pipeline = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
rfc = RFC()

**Expert** features:

In [37]:
scores_wisdm['lr_expert'] = get_score(df_expert_wisdm, lr_pipeline, parameters_lr)
scores_wisdm['svm_expert'] = get_score(df_expert_wisdm, svc_pipeline, parameters_svm)
scores_wisdm['rf_expert'] = get_score(df_expert_wisdm, rfc, parameters_rf)

In [38]:
scores_uschad['lr_expert'] = get_score(df_expert_uschad, lr_pipeline, parameters_lr)
scores_uschad['svm_expert'] = get_score(df_expert_uschad, svc_pipeline, parameters_svm)
scores_uschad['rf_expert'] = get_score(df_expert_uschad, rfc, parameters_rf)

From **autoregression model** features:

In [39]:
n = 20

In [40]:
scores_wisdm['lr_ar_' + str(n)] = get_score(df_ar_wisdm, lr_pipeline, parameters_lr)
scores_wisdm['svm_ar_' + str(n)] = get_score(df_ar_wisdm, svc_pipeline, parameters_svm)
scores_wisdm['rf_ar_' + str(n)] = get_score(df_ar_wisdm, rfc, parameters_rf)

In [41]:
scores_uschad['lr_ar_' + str(n)] = get_score(df_ar_uschad, lr_pipeline, parameters_lr)
scores_uschad['svm_ar_' + str(n)] = get_score(df_ar_uschad, svc_pipeline, parameters_svm)
scores_uschad['rf_ar_' + str(n)] = get_score(df_ar_uschad, rfc, parameters_rf)

From **spectrum analysis** features:

In [42]:
n = 20

In [43]:
scores_wisdm['lr_ssa_' + str(n)] = get_score(df_ssa_wisdm, lr_pipeline, parameters_lr)
scores_wisdm['svm_ssa_' + str(n)] = get_score(df_ssa_wisdm, svc_pipeline, parameters_svm)
scores_wisdm['rf_ssa_' + str(n)] = get_score(df_ssa_wisdm, rfc, parameters_rf)

In [44]:
scores_uschad['lr_ssa_' + str(n)] = get_score(df_ssa_uschad, lr_pipeline, parameters_lr)
scores_uschad['svm_ssa_' + str(n)] = get_score(df_ssa_uschad, svc_pipeline, parameters_svm)
scores_uschad['rf_ssa_' + str(n)] = get_score(df_ssa_uschad, rfc, parameters_rf)

## Results 

In [74]:
results_wisdm = pd.DataFrame.from_dict(scores_wisdm, orient='index').sort_index()
results_wisdm.columns = ['all'] + list(set(data_wisdm['activity']))
results_wisdm.to_csv("results_wisdm_norm.csv")

results_uschad = pd.DataFrame.from_dict(scores_uschad, orient='index').sort_index()
results_uschad.columns = ['all'] + list(set(data_uschad['activity']))
results_uschad.to_csv("results_uschad_norm.csv")

In [75]:
results_wisdm

,all,Standing,Walking,Upstairs,Sitting,Jogging,Downstairs
lr_ar_20,0.908373,0.983161,0.960247,0.950254,0.980015,0.987417,0.955652
lr_expert,0.854433,0.991149,0.907448,0.908867,0.990069,0.983963,0.927371
lr_ssa_20,0.842005,0.996268,0.861280,0.907016,0.995929,0.987695,0.935821
rf_ar_20,0.928697,0.988065,0.968697,0.956207,0.988157,0.989915,0.966353
rf_expert,0.932953,0.998088,0.958304,0.959322,0.997348,0.987849,0.964996
rf_ssa_20,0.924842,0.996638,0.947880,0.959753,0.996330,0.986307,0.962776
svm_ar_20,0.950594,0.983315,0.981311,0.979368,0.983932,0.993924,0.979337
svm_expert,0.927895,0.991118,0.963115,0.957533,0.991025,0.989823,0.963177
svm_ssa_20,0.953338,0.997995,0.976407,0.973385,0.997348,0.988497,0.973045


In [76]:
results_uschad

,all,Standing,Elevator-up,Walking-forward,Sitting,Walking-downstairs,Sleeping,Elevator-down,Walking-upstairs,Jumping,Walking-right,Walking-left,Running
lr_ar_20,0.650876,0.936495,0.944395,0.865972,0.952501,0.934998,0.979041,0.943446,0.950171,0.990631,0.904425,0.906667,0.993010
lr_expert,0.668135,0.936711,0.940773,0.866197,0.978708,0.950739,1.000000,0.939608,0.943857,0.992648,0.905335,0.894195,0.987499
lr_ssa_20,0.636750,0.923749,0.927332,0.885198,0.939726,0.931229,0.994459,0.941674,0.934635,0.996505,0.908057,0.896407,0.994528
rf_ar_20,0.703025,0.943651,0.946559,0.892785,0.961654,0.957709,0.981654,0.947636,0.948791,0.986314,0.921811,0.925688,0.991796
rf_expert,0.871229,0.975330,0.950426,0.967744,0.991826,0.986177,0.999902,0.949995,0.980822,0.998257,0.973010,0.972227,0.996740
rf_ssa_20,0.840147,0.966618,0.946432,0.962986,0.977230,0.980000,0.995467,0.947068,0.977592,0.997866,0.964023,0.967998,0.997014
svm_ar_20,0.654606,0.942770,0.943054,0.871630,0.960666,0.930269,0.976662,0.943045,0.947234,0.989271,0.901419,0.906853,0.996339
svm_expert,0.797279,0.945022,0.934087,0.945717,0.978326,0.980323,0.999491,0.932658,0.977425,0.997827,0.956828,0.950514,0.996339
svm_ssa_20,0.822320,0.974596,0.940108,0.971375,0.985179,0.980460,0.999540,0.939393,0.983035,0.970005,0.974440,0.973784,0.952726


In [77]:
df_all_wisdm = df_expert_wisdm.join(df_ar_wisdm.iloc[:, 1:])
df_all_wisdm = df_all_wisdm.join(df_ssa_wisdm.iloc[:, 1:])

df_all_uschad = df_expert_uschad.join(df_ar_uschad.iloc[:, 1:])
df_all_uschad = df_all_uschad.join(df_ssa_uschad.iloc[:, 1:])

print(df_all_wisdm.shape, df_all_uschad.shape)

((4321, 164), (13620, 164))


In [49]:
scores_wisdm['lr_all'] = get_score(df_all_wisdm, lr_pipeline, parameters_lr)
scores_wisdm['svm_all'] = get_score(df_all_wisdm, svc_pipeline, parameters_svm)
scores_wisdm['rf_all'] = get_score(df_all_wisdm, rfc, parameters_rf)

scores_uschad['lr_all'] = get_score(df_all_uschad, lr_pipeline, parameters_lr)
scores_uschad['svm_all'] = get_score(df_all_uschad, svc_pipeline, parameters_svm)
scores_uschad['rf_all'] = get_score(df_all_uschad, rfc, parameters_rf)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

,0_x,0_y,0_z,1_x,1_y,1_z,2_x,2_y,2_z,3_x,...,eigv_9_x,eigv_9_y,eigv_9_z,intercept_x,intercept_y,intercept_z,mean,std_x,std_y,std_z
0,0.015,0.050,0.020,0.135,0.050,0.025,0.000,0.00,0.09,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,0.245617,0.003070,0.004086,0.005414
1,0.010,0.025,0.005,0.000,0.025,0.040,0.075,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,0.246386,0.002650,0.003595,0.003959
2,0.085,0.010,0.015,0.000,0.000,0.000,0.380,0.04,0.08,0.43,...,NaN,NaN,NaN,NaN,NaN,NaN,0.247450,0.002902,0.002664,0.003084
3,0.035,0.005,0.015,0.000,0.045,0.185,0.340,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,0.250888,0.002452,0.004422,0.003975
4,0.005,0.035,0.005,0.050,0.230,0.040,0.000,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,0.250704,0.002948,0.004118,0.004183


In [53]:
df_expert_uschad.head()

,activity,avg_x,avg_y,avg_z,std_x,std_y,std_z,abs_x,abs_y,abs_z,...,6_z,7_x,7_y,7_z,8_x,8_y,8_z,9_x,9_y,9_z
0,0,0.968170,-0.015659,-0.215660,0.003070,0.004086,0.005414,0.002483,0.002744,0.003744,...,0.070,0.00,0.000,0.02,0.065,0.035,0.015,0.0,0.0,0.0
1,0,0.968547,-0.014863,-0.214525,0.002650,0.003595,0.003959,0.002266,0.002604,0.003111,...,0.215,0.39,0.000,0.00,0.000,0.050,0.125,0.0,0.0,0.0
2,0,0.969028,-0.013271,-0.213409,0.002902,0.002664,0.003084,0.002420,0.002227,0.002598,...,0.000,0.01,0.525,0.09,0.000,0.000,0.000,0.0,0.0,0.0
3,0,0.969457,-0.007118,-0.209675,0.002452,0.004422,0.003975,0.002043,0.003722,0.003342,...,0.375,0.10,0.000,0.00,0.000,0.115,0.130,0.0,0.0,0.0
4,0,0.969818,-0.007371,-0.210334,0.002948,0.004118,0.004183,0.002236,0.003313,0.003450,...,0.320,0.00,0.000,0.00,0.135,0.025,0.110,0.0,0.0,0.0


In [54]:
df_ar_uschad.head()

,activity,intercept_x,coef_0_x,coef_1_x,coef_2_x,coef_3_x,coef_4_x,coef_5_x,coef_6_x,coef_7_x,...,coef_10_z,coef_11_z,coef_12_z,coef_13_z,coef_14_z,coef_15_z,coef_16_z,coef_17_z,coef_18_z,coef_19_z
0,0,1.051263,-0.027841,-0.003548,-0.056764,0.032787,-0.040460,-0.007357,-0.019385,-0.067670,...,-0.141857,-0.043810,0.076310,0.076808,0.027878,-0.177782,-0.123808,0.128057,0.156536,0.748830
1,0,1.607836,-0.094231,-0.001729,-0.097072,0.044802,0.029601,-0.017225,-0.080312,-0.053346,...,-0.000590,-0.094244,0.044033,0.050491,-0.085154,0.007570,0.068591,-0.078060,0.249683,0.566482
2,0,2.256061,-0.170195,0.023232,-0.097438,0.053526,0.008247,-0.256334,-0.063838,0.040628,...,0.029612,-0.149447,0.099807,0.096182,-0.061418,-0.058666,-0.117114,0.082011,0.215266,0.520579
3,0,0.866865,-0.118681,-0.078609,0.134309,0.118477,0.015025,-0.015247,-0.114312,-0.027888,...,-0.133565,-0.022016,-0.085181,0.018978,-0.032694,0.073873,-0.063996,0.083845,0.287215,0.509311
4,0,1.059768,0.000617,-0.105476,-0.013691,-0.002752,0.119131,0.007464,-0.069395,-0.143768,...,-0.049980,0.278623,-0.058478,0.013521,-0.165579,0.088449,-0.048339,-0.026495,0.219643,0.686094
